<a href="https://colab.research.google.com/github/samuelcampione/CNN_Techniques_for_Image_Classification_using_CIFAR10/blob/main/notebooks/week-01/W1_HW1_SQL_NinjaTraining_(dla_tutorial)_withSolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# W1 Homework 1: SQL fundamentals, outcome and message tables (2023-01)

(c) Johannes Eichstaedt & the World Well-Being Project, 2023.

## Setup Colab

As discussed in the tutorials, we begin by setting up the Colab environment. The next couple of subsections do this for you.

### Install packages and download data

We first install the necessary packages and then download the dataset.

In [ ]:
# installing DLATK and necessary packages
!git clone -b psych290 https://github.com/dlatk/dlatk.git
!pip install dlatk/
!pip install jupysql

!git clone https://github.com/CompPsychology/psych290_data.git

fatal: destination path 'dlatk' already exists and is not an empty directory.
Processing ./dlatk
  Preparing metadata (setup.py) ... done
  Created wheel for dlatk: filename=dlatk-1.3.1-py3-none-any.whl size=35635831 sha256=f9707263aede4212d3a33605b4c0d98358c0b7220bad2a5ec613580afbf9340d
  Stored in directory: /tmp/pip-ephem-wheel-cache-m2dvtssy/wheels/cc/c9/65/e1ecc64bac68518c07b286fe86921aa938e11a0c3a87d8ff93
Successfully built dlatk
  Attempting uninstall: dlatk
    Found existing installation: dlatk 1.3.1
    Uninstalling dlatk-1.3.1:
      Successfully uninstalled dlatk-1.3.1
Cloning into 'psych290_data'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 11 (delta 0), reused 8 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (11/11), 13.84 MiB | 17.14 MiB/s, done.


### Load data

We then create a `username` variable that is required to connect to the database.

In [ ]:
username = "shashanka"

We then load the downloaded data into a database named `[username].db` in the `sqlite_data` folder.

In [ ]:
# this loads the required package -- similar to library() function in R
import os
from dlatk.tools.importmethods import csvToSQLite

# creates stores the complete path to the database -- sqlite_data/[username].db
database = os.path.join("sqlite_data", username)

# this imports CSVs into tables in this database
csvToSQLite(
    "psych290_data/dla_tutorial/msgs.csv",
    database,
    "dla_tutorial_msgs"
)

csvToSQLite(
    "psych290_data/dla_tutorial/blog_outcomes.csv",
    database,
    "dla_tutorial_outcomes"
)

SQL Query: CREATE TABLE dla_tutorial_msgs (message_id INT, user_id INT, date VARCHAR(31), created_time VARCHAR(31), message LONGTEXT);


Importing data, reading psych290_data/dla_tutorial/msgs.csv file
Reading 10000 rows into the table...
Reading 10000 rows into the table...
Reading 10000 rows into the table...
Reading remaining 1674 rows into the table...
Importing data, reading psych290_data/dla_tutorial/blog_outcomes.csv file
Reading remaining 1000 rows into the table...


SQL Query: CREATE TABLE dla_tutorial_outcomes (user_id INT, gender INT, age INT, occu VARCHAR(31), sign VARCHAR(15), is_indunk INT, is_student VARCHAR(7), is_education VARCHAR(7), is_technology VARCHAR(7));


### Setup database connection

Finally, we establish a connection with the database.

In [ ]:
# loads the %%sql extension
%load_ext sql

# connects the extension to the database
from sqlalchemy import create_engine
engine = create_engine(f"sqlite:///sqlite_data/{username}.db?charset=utf8mb4")
%sql engine

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Recap

### List all the tables.

After connecting to a specific database, list all the tables inside it using a `%sqlcmd` statement. For a more readable output, you can save the output to a variable and print it.

In [ ]:
result = %sqlcmd tables
print(result)

+-----------------------+
|          Name         |
+-----------------------+
|   dla_tutorial_msgs   |
| dla_tutorial_outcomes |
+-----------------------+


This homework will use both `dla_tutorial_msgs` and `dla_tutorial_outcomes` tables.

### Now, let's view schema of these tables.

Remember, `PRAGMA` and  `%sqlcmd` are statements that help run meta commands like looking at table details and database properties.

In [ ]:
%%sql

PRAGMA table_info(dla_tutorial_msgs);

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

cid,name,type,notnull,dflt_value,pk
0,message_id,INT,0,None,0
1,user_id,INT,0,None,0
2,date,VARCHAR(31),0,None,0
3,created_time,VARCHAR(31),0,None,0
4,message,LONGTEXT,0,None,0


In [ ]:
%%sql

PRAGMA table_info(dla_tutorial_outcomes);

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

cid,name,type,notnull,dflt_value,pk
0,user_id,INT,0,None,0
1,gender,INT,0,None,0
2,age,INT,0,None,0
3,occu,VARCHAR(31),0,None,0
4,sign,VARCHAR(15),0,None,0
5,is_indunk,INT,0,None,0
6,is_student,VARCHAR(7),0,None,0
7,is_education,VARCHAR(7),0,None,0
8,is_technology,VARCHAR(7),0,None,0


### They are missing indices. Let's add indices on the main column that connects the table.

Since the `user_id` columns have the same name in both the tables, we'll use different names for their respective indices to avoid collision.

In [ ]:
%%sql

CREATE INDEX msg_user_id ON dla_tutorial_msgs (user_id);
CREATE INDEX message_id ON dla_tutorial_msgs (message_id);

CREATE INDEX outcome_user_id ON dla_tutorial_outcomes (user_id);

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

++
||
++
++

## Questions

### 0) Select 10 random rows from both tables.

Look at the output, familiarize yourself with it a little bit. 👀 The messages in the msgs table are blog posts. Skim one.

**Answer:**

In [ ]:
%%sql

SELECT *
FROM dla_tutorial_outcomes
ORDER BY RANDOM()
LIMIT 10;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

user_id,gender,age,occu,sign,is_indunk,is_student,is_education,is_technology
3375539,0,17,Student,Aries,0,1,NA,NA
4061665,0,25,Advertising,Leo,0,0,NA,NA
1987893,1,38,indUnk,Capricorn,1,NA,NA,NA
3551359,0,23,Communications-Media,Cancer,0,0,NA,NA
3431501,0,15,indUnk,Sagittarius,1,NA,NA,NA
3856641,0,27,Arts,Aries,0,0,NA,NA
3509141,1,16,Student,Cancer,0,1,NA,NA
4292654,1,33,Banking,Libra,0,0,NA,NA
3835812,1,47,Non-Profit,Pisces,0,0,NA,NA
3655970,0,17,Religion,Sagittarius,0,0,NA,NA


In [ ]:
%%sql

SELECT *
FROM dla_tutorial_msgs
ORDER BY RANDOM()
LIMIT 10;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

+------------+---------+----------------+---------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 1) Calculate the minimum, maximum and average age for both genders.

**Answer:**

In [ ]:
%%sql

SELECT gender, MIN(age) AS min_age, MAX(age) AS max_age, AVG(age) AS avg_age
FROM dla_tutorial_outcomes
GROUP BY gender;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

gender,min_age,max_age,avg_age
0,13,48,23.1195219123506
1,13,48,23.337349397590362


### 2) Calculate the average age for every occupation

**Answer:**

In [ ]:
%%sql

SELECT occu, AVG(age) AS avg_age
FROM dla_tutorial_outcomes
GROUP BY occu;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

occu,avg_age
Accounting,24.5
Advertising,25.5
Agriculture,17.0
Architecture,28.333333333333332
Arts,23.0625
Automotive,30.5
Banking,27.875
Biotech,24.5
BusinessServices,29.5
Chemicals,15.0


### 3) What's the average age difference between Libras and Leos?

We need a `GROUP BY` and `AVG` but we don't need to that for all groups, just the ones with care about.

**Hint:** You can do just get the average age for Libras and Leos, and then subtract them from one another in a separate command.

**Answer:**

In [ ]:
%%sql

SELECT sign, AVG(age) AS avg_age
FROM dla_tutorial_outcomes
WHERE sign IN ('Libra','Leo')
GROUP BY sign;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

sign,avg_age
Leo,22.772151898734176
Libra,24.216867469879517


In [ ]:
%%sql

SELECT (24.2169 - 22.7722) AS avg_difference;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

avg_difference
1.4446999999999974


### 4) How many blog posts do we have from authors from a given age?

i.e., how many from age 17, 18, etc.

**Hint:** First create a table with number of messages per user, then join on the extra info that you need.

**Answer:**

In [ ]:
%%sql

DROP TABLE IF EXISTS n_msgs_per_user;

CREATE TABLE n_msgs_per_user AS
SELECT user_id, COUNT(*) AS n_msgs
FROM dla_tutorial_msgs
GROUP BY user_id;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

++
||
++
++

In [ ]:
%%sql

SELECT *
FROM n_msgs_per_user
ORDER BY n_msgs DESC
LIMIT 10;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

user_id,n_msgs
942828,2068
3523319,1052
1234212,643
317581,619
664485,566
1694057,491
979795,425
1488330,403
671748,351
3196705,347


We then join the tables `dla_tutorial_outcomes` with `n_msgs_per_user` and compute the sum of number of messages grouped by age.

In [ ]:
%%sql

SELECT age, SUM(B.n_msgs) AS n_msgs
FROM dla_tutorial_outcomes AS A INNER JOIN n_msgs_per_user AS B ON A.user_id = B.user_id
GROUP BY age
ORDER BY age;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

age,n_msgs
13,783
14,1417
15,2093
16,2255
17,3588
23,2790
24,4356
25,3304
26,2699
27,2979


### 5) How many blog posts do we have from authors in an industry?

Here we use the table we made earlier `n_msgs_per_user`. We join it to `dla_tutorial_outcomes` and compute sum of number of messages grouped by occupation.

**Answer:**

In [ ]:
%%sql

SELECT occu, SUM(B.n_msgs) AS n_msgs
FROM dla_tutorial_outcomes AS A INNER JOIN n_msgs_per_user AS B ON A.user_id = B.user_id
GROUP BY occu
ORDER BY n_msgs DESC;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

occu,n_msgs
indUnk,11548
Student,6650
Technology,2203
Education,1717
Communications-Media,1625
Internet,1599
Arts,1280
Government,661
Science,627
Non-Profit,584


### 6) How many blog posts do we have from a given year?

**Hint:** remember the **YEAR(sql_timeformat)** command.

**Answer:**

In [ ]:
%%sql

SELECT strftime("%Y", created_time) AS yyyy, COUNT(*) AS n_msgs
FROM dla_tutorial_msgs
GROUP by yyyy
ORDER by yyyy;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

yyyy,n_msgs
2001,331
2002,710
2003,5031
2004,25600
2005,1
2006,1


### 7) How many blog posts do we have from a given year only written by  Tauruses?

**Answer:**

In [ ]:
%%sql

SELECT strftime("%Y", created_time) AS yyyy, COUNT(*) AS n_msgs
FROM dla_tutorial_msgs AS a, dla_tutorial_outcomes AS b
WHERE a.user_id = b.user_id AND b.sign = "Taurus"
GROUP by yyyy
ORDER by yyyy;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

yyyy,n_msgs
2001,2
2002,116
2003,1559
2004,2660


### 8) Should you do something to make the last queries for questions 1 and 2 run faster? If so, give the query

In our queries, we are partitioning columns with **GROUP BY**. So, it may help _index_ these columns.

**Answer:**

In [ ]:
%%sql

CREATE INDEX age ON dla_tutorial_outcomes (age);
CREATE INDEX gender ON dla_tutorial_outcomes (gender);
CREATE INDEX occu ON dla_tutorial_outcomes (occu);

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

++
||
++
++

### 9) Can you pull out 10 blog posts that mention the word "boys"?

**Answer:**

In [ ]:
%%sql

SELECT *
FROM dla_tutorial_msgs
WHERE message LIKE '%boys%'
ORDER BY RANDOM() LIMIT 10;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

+------------+---------+------------------+---------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 10) Can you make the last query run faster?

**Hint:** It's a text field. you need a FULLTEXT...something. Or convert ...a thing... to VARCHAR first.

**Answer:**

In [ ]:
%%sql

ALTER TABLE msgs
ADD FULLTEXT index message (message);

**Extended answer for later:** Another approach is to re-arrange our data to be able to find messages by words faster. We can transform the data by going over every message, splitting it into words (tokens in NLP jargon), making a table, say tokens_table, of (word, message_id). We can then create an _index_ on the words so that when we receive a new word to search, we lookup in table and return a set of message_id. This will be much faster to search, but require more processing and indexing upfront.

DLATK will go that route for us.

If you are familiar with hash-tables, this is something like that giving us very fast lookup times.

The difference is (24.2169 - 22.7722). Small difference now, but believe us, it will make all the difference in the world.

### 11) (EXTRA CREDIT) How many blog posts do we have from a given month (i.e., from Feb of 2001, and so forth)?

**Hint:** **ORDER BY** is not the only thingy that can take two variables.

**Answer:**

In [ ]:
%%sql

SELECT strftime("%Y", created_time) AS yyyy, strftime("%m", created_time) AS mm, COUNT(*) AS n_msgs
FROM dla_tutorial_msgs
GROUP BY yyyy, mm
ORDER BY yyyy, mm;

Running query in 'sqlite:///sqlite_data/shashanka.db?charset=utf8mb4'

yyyy,mm,n_msgs
2001,02,1
2001,04,1
2001,08,77
2001,09,100
2001,10,100
2001,11,33
2001,12,19
2002,01,28
2002,02,19
2002,03,13
